In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
#import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
#from git import Repo
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser

In [6]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [1]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit10.csv')

NameError: name 'pd' is not defined

In [8]:
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
0,Hive,https://github.com/apache/hive,release-2.0.1,"Commit(sha=""e3cfeebcefe9a19c5055afdcbb00646908...",2016-05-04 04:02:05
1,Hive,https://github.com/apache/hive,release-2.0.0,"Commit(sha=""7f9f1fcb8697fb33f0edc2c391930a3728...",2016-02-10 01:53:25
2,Hive,https://github.com/apache/hive,release-1.2.1,"Commit(sha=""243e7c1ac39cb7ac8b65c5bc6988f5cc31...",2015-06-19 08:49:55
3,Hive,https://github.com/apache/hive,release-1.2.0,"Commit(sha=""7f237de447bcd726bb3d0ba332cbb733f3...",2015-05-15 00:59:07
4,Hive,https://github.com/apache/hive,release-1.1.1,"Commit(sha=""3e8d832a1a8e2b12029adcb55862cf0400...",2015-05-14 22:09:06


In [9]:
#project_releases = project_releases[project_releases['project_name'] == 'Okhttp'].head()

In [10]:
project_releases.head(10)

,project_name,project_link,version_name,commit,timestamp
0,Hive,https://github.com/apache/hive,release-2.0.1,"Commit(sha=""e3cfeebcefe9a19c5055afdcbb00646908...",2016-05-04 04:02:05
1,Hive,https://github.com/apache/hive,release-2.0.0,"Commit(sha=""7f9f1fcb8697fb33f0edc2c391930a3728...",2016-02-10 01:53:25
2,Hive,https://github.com/apache/hive,release-1.2.1,"Commit(sha=""243e7c1ac39cb7ac8b65c5bc6988f5cc31...",2015-06-19 08:49:55
3,Hive,https://github.com/apache/hive,release-1.2.0,"Commit(sha=""7f237de447bcd726bb3d0ba332cbb733f3...",2015-05-15 00:59:07
4,Hive,https://github.com/apache/hive,release-1.1.1,"Commit(sha=""3e8d832a1a8e2b12029adcb55862cf0400...",2015-05-14 22:09:06
5,Hive,https://github.com/apache/hive,release-1.1.0,"Commit(sha=""e60744d017ef79f1b17f474c0b969d4ca5...",2015-03-09 02:37:05
6,Hive,https://github.com/apache/hive,release-1.0.1,"Commit(sha=""73b600dc79ba8a9a32078a2ea0eb8ae3df...",2015-05-14 21:54:45
7,Hive,https://github.com/apache/hive,release-1.0.0,"Commit(sha=""697aecadc3ba62bc11f3ba0a6c8522daee...",2015-02-03 20:35:46
8,Hive,https://github.com/apache/hive,release-0.14.0,"Commit(sha=""70eea944afaf5f64b8939b2b3a0d8fdd33...",2014-11-12 20:35:15
9,Hive,https://github.com/apache/hive,release-0.13.1,"Commit(sha=""89bb2e8c34386b43efa37ecc6918a50c4b...",2014-06-05 21:03:32


In [ ]:
finished_project_arr = []
with open('finished_projects.txt','r') as finished_project:
    for line in finished_project:
        finished_project_arr.append(line.split('\n')[0])
    finished_project.close()
    
for project_name in project_releases['project_name'].unique():
    if project_name not in finished_project_arr:
        current_project = pd.DataFrame(project_releases[project_releases['project_name'] == project_name])
        current_project['rank'] = current_project['timestamp'].rank()
        current_project = current_project.sort_values(by=['rank'], ascending=False)
        current_project['previous_version'] = current_project['version_name'].shift(-1)
        current_project['previous_version_timestamp'] = current_project['timestamp'].shift(-1)
        current_project['previous_version_timestamp'].fillna(current_project['timestamp'], inplace=True)
        current_project['timestamp']= pd.to_datetime(current_project['timestamp'])
        current_project['previous_version_timestamp']= pd.to_datetime(current_project['previous_version_timestamp'])
        current_project['timestamp_diff'] = current_project['timestamp'] - current_project['previous_version_timestamp']
        current_project['timestamp_diff_hours'] = current_project['timestamp_diff'].astype('timedelta64[h]')
        current_project.tail()
        
        
        ### Download dataset
        try:
            os.mkdir('raw_sourcecode/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            command = 'cd E:/SoftwareRemodularization/raw_sourcecode/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']

            #print(command)
            os.system(command)

            command = 'git clone ' + row[1]['project_link'] +  ' E:/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name + '_' + row[1]['version_name']

            #print(command)
            os.system(command)
            commit = row[1]['commit'].replace('Commit(sha="','')
            commit = commit.replace('")','')
            command = 'cd E:/SoftwareRemodularization/raw_sourcecode/' + project_name + '/' + project_name +'_' + row[1]['version_name'] + ' & git checkout ' + commit
            #print(command)
            os.system(command)
        
        try:
            os.mkdir('raw_depends/' + project_name)
        except:
            pass
        for row in current_project.iterrows():
            #command = 'cd E:/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
            #os.system(command)
            #print(command)
            command = 'cd E:/SoftwareRemodularization/depends-0.9.2 & ' + 'java -jar depends.jar java E:/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name']+  ' ../raw_depends/' + project_name + "/" + project_name + '_' + row[1]['version_name'] 
            #print(command)
            os.system(command)

        first_tag = current_project.head(1)['version_name'].values[0]
        last_tag= current_project.tail(1)['version_name'].values[0]

        #print(first_tag)
        #print(last_tag)

        try:
            os.mkdir('raw_refactoringMiner/' + project_name)
        except:
            pass
        
        try:
            for row in current_project.iterrows():
                #command = 'cd E:/SoftwareRemodularization/raw_depends/' + project_name +' & mkdir ' + project_name + '_' +row[1]['version_name']
                #os.system(command)
                #print(command)
                command = 'cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & ' + 'RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/'  + project_name + '/' + project_name + '_' + row[1]['version_name'] + ' ' + row[1]['previous_version'] + ' ' + row[1]['version_name'] + ' -json ' + 'E:/SoftwareRemodularization/raw_refactoringMiner/' +project_name + '/'+ project_name + '_' + row[1]['version_name'] + '.json'
                print(command)
                os.system(command)
        except:
            pass
       
        
        main_data_arr = []
        
        for row in current_project.iterrows():
            try:
                refactoring_miner_filename = f'raw_refactoringMiner/{project_name}/' + project_name + '_' + row[1]['version_name'] + '.json'
                depends_filename = f'raw_depends/{project_name}/' + project_name + '_' + row[1]['version_name'] + '.json'
                latest_commit = row[1]['commit'].replace('Commit(sha="','')
                latest_commit = latest_commit.replace('")','')
                print(latest_commit)
                print(refactoring_miner_filename)

                print(depends_filename)
                with open(depends_filename) as f:
                    depends_results = json.load(f)
                    index  = 0
                    file_dict = {}
                    for value in depends_results['variables']:
                        file_path = value.replace('\\','/')
                        string = f"E:/SoftwareRemodularization/raw_sourcecode/{project_name}/{project_name}_{row[1]['version_name']}/"
                        #print(string)
                        file_path = file_path.replace(string, '')
                        #var_array.append([index, value.replace('\\','/')])
                        #var_array.append([index,file_path])
                        file_dict[file_path] = index
                        #var_array.append(file_path)
                        #var_array.append([index, value.split('\\')[-1]])
                        #print(index, value)
                        index += 1

                    #var_df = pd.DataFrame(var_array)
                    #var_df.columns = ['index_val', 'name']
                    #var_df = var_df.set_index('name')

                    feature_list = {}
                    feature_index = 2
                    for element in depends_results['cells']:
                        #print(element)
                        try:
                            for a in element['values']:
                                if a not in feature_list:
                                    feature_list[a] = feature_index
                                    feature_index += 1
                                #print(a['Call'])


                        except:
                            pass

                    feature_arr = []
                    for element in depends_results['cells']:
                        #print(array)
                        array = [0] * (len(feature_list) + 2)
                        values = dict(element['values'])
                        #print(element)
                        array[0] = element['src']
                        array[1] = element['dest']
                        for feature in feature_list:
                            try:
                                value = values[feature]
                                array[feature_list[feature]] = value
                            except:
                                pass
                        #print(array)
                        feature_arr.append(array)

                    feature_df = pd.DataFrame(feature_arr)
                    col_names = ['src', 'dest']
                    for element in feature_list:
                        col_names.append(element)
                    feature_df.columns = col_names

                    feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']

                G = nx.Graph()
                for index, row in feature_df.iterrows():
                    G.add_edge(row['src'], row['dest'], weight=row['sum'])

                sum_dependency_df = nx.to_pandas_adjacency(G)
                sum_dependency_df['sum'] = sum_dependency_df.sum(axis=1)
                #sum_dependency_df
                final_dependency_df = sum_dependency_df['sum']

                try:
                    with open(refactoring_miner_filename) as f:
                        print(refactoring_miner_filename, 'stuck here')
                        refactoring_miner = json.load(f)

                        for i in refactoring_miner['commits']:

                            if len(i['refactorings']) != 0:
                                commit_compared_with = i['sha1']
                                file_arr = []
                                num_line_affected = 0
                                num_dependency = 0
                                actual_num_of_classes_touched = 0
                                for refactor in i['refactorings']:
                                    #print(refactor)
                                    for file in refactor['rightSideLocations']:

                                        #file_arr.append(file['filePath'])
                                        #print(file['filePath'], 'here')
                                        #print(file_dict[file['filePath']])
                                        actual_num_of_classes_touched += 1
                                        try:
                                            num_dependency += final_dependency_df.iloc[file_dict[file['filePath']]]
                                            num_line_affected += file['endLine'] - file['startLine']
                                        except:
                                            pass

                                    for file in refactor['leftSideLocations']:

                                        #file_arr.append(file['filePath'])
                                        #print(file['filePath'], 'here')
                                        #print(file_dict[file['filePath']])
                                        #actual_num_of_classes_touched += 1
                                        try:
                                            num_dependency += final_dependency_df.iloc[file_dict[file['filePath']]]
                                            num_line_affected += file['endLine'] - file['startLine']
                                        except:
                                            pass

                                if num_dependency != 0 and num_line_affected != 0:
                                    print('added to main_data_arr')
                                    main_data_arr.append([latest_commit, commit_compared_with, num_dependency, num_line_affected, actual_num_of_classes_touched])
                
                except Exception as e:
                    print(e)

            except Exception as e:
                print(e)

        effort_data_df = pd.DataFrame(main_data_arr)
        effort_data_df.columns = ['latest_commit', 'commit_compared_with', 'num_dependency', 'num_line_affected', 'actual_num_of_classes_touched']

        latest_commit_arr = []
        commit_compared_with_arr = []
        
        g = Github(github_oauth, timeout=3000)
        repo = g.get_repo(current_project.iloc[0]['project_link'].replace('https://github.com/',''))
        
        for row in effort_data_df.iterrows():
            sha = row[1]['latest_commit']
            commit = repo.get_commit(sha=sha)
            print(commit)
            date = commit.commit.author.date
            print(date)
            latest_commit_arr.append(date)

            sha = row[1]['commit_compared_with']
            commit = repo.get_commit(sha=sha)
            date = commit.commit.author.date
            commit_compared_with_arr.append(date)

        effort_data_df['latest_commit_date'] = latest_commit_arr
        effort_data_df['commit_compared_with_date'] = commit_compared_with_arr

        effort_data_df.to_csv('Effort_Estimation_Results/'+ project_name + '.csv', index=False)
        
        
        with open('finished_projects.txt','a+') as finished_project:
            finished_project.write(project_name + '\n')
            finished_project.close()

cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_21.1.4 21.1.3 21.1.4 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_21.1.4.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_21.1.3 21.1.2 21.1.3 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_21.1.3.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_21.1.2 21.1.1 21.1.2 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_21.1.2.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_21.1.1 21.1.0 21.1.1 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_21.1.1.json
cd E:/SoftwareRemodulari

cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_7.0.0 6.3.5 7.0.0 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_7.0.0.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_6.3.5 6.3.4 6.3.5 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_6.3.5.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_6.3.4 6.3.3 6.3.4 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_6.3.4.json
cd E:/SoftwareRemodularization/RefactoringMiner-2.1.0/bin & RefactoringMiner -bt E:/SoftwareRemodularization/raw_sourcecode/Dbeaver/Dbeaver_6.3.3 6.3.2 6.3.3 -json E:/SoftwareRemodularization/raw_refactoringMiner/Dbeaver/Dbeaver_6.3.3.json
cd E:/SoftwareRemodularization/Refactori

raw_refactoringMiner/Dbeaver/Dbeaver_21.0.2.json stuck here
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
8cff034373319ebb496de5ca6dff7e5ba619aa7a
ra

raw_refactoringMiner/Dbeaver/Dbeaver_7.2.5.json stuck here
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
d6c41e77e08cd100eebf2

raw_refactoringMiner/Dbeaver/Dbeaver_7.1.2.json stuck here
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
7e60a77cf76c19fdaf9e87786243054071b63cf1
raw_refactoringMiner/Dbeaver/Dbeaver_7.1.1.json
raw_depends/Dbeaver/Dbeaver_7.1.1.json
raw_refactoringMiner/Dbeaver/Dbeaver_7.1.1.json stuck here
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data_arr
added to main_data

...

Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e337322c0743345ebdb46c507727")
2020-08-02 15:53:26
Commit(sha="419770e906c0e

Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="7e60a77cf76c19fdaf9e87786243054071b63cf1")
2020-06-21 16:52:27
Commit(sha="0372e776e7079407b932c4bb21dc2557ca194333")
2020-05-31 17:27:48
Commit(sha="0372e776e7079407b932c4bb21dc2557ca194333")
2020-05-31 17:27:48
Commit(sha="0372e776e7079

Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a17b834d7b6204010417074b3171c")
2020-04-18 12:18:18
Commit(sha="266910c7749a1

Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="6e41bc0763bb96ed6bb99383f860108548101ad1")
2020-02-29 18:16:19
Commit(sha="a7fefed90cfc995e64889537b2bca324796ca5e4")
2020-02-15 12:41:00
Commit(sha="a7fefed90cfc995e64889537b2bca324796ca5e4")
2020-02-15 12:41:00
Commit(sha="a7fefed90cfc995e64889537b2bca324796ca5e4")
2020-02-15 12:41:00
Commit(sha="a7fefed90cfc995e64889537b2bca324796ca5e4")
2020-02-15 12:41:00
Commit(sha="a7fefed90cfc995e64889537b2bca324796ca5e4")
2020-02-15 12:41:00
Commit(sha="a7fefed90cfc9

Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f04a846a57fc4a5b78b7c5c8ac81c")
2020-01-17 17:42:31
Commit(sha="235ae8dc799f0

In [32]:
num_dependency += final_dependency_df[file_dict[file['filePath']]]

""


In [50]:
main_data_arr

[]

In [38]:
file_dict[file['filePath']]

1

In [45]:
final_dependency_df

157.0      4.0
155.0      4.0
30.0     321.0
41.0       9.0
51.0      86.0
         ...  
99.0       4.0
83.0      16.0
71.0      18.0
72.0      18.0
75.0      29.0
Name: sum, Length: 68, dtype: float64

In [44]:
final_dependency_df.iloc[1]

4.0

In [46]:
final_dependency_df.iloc[file_dict[file['filePath']]]

4.0

In [16]:
current_project

,project_name,project_link,version_name,commit,timestamp,rank,previous_version,previous_version_timestamp,timestamp_diff,timestamp_diff_hours
897,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",2021-01-30 19:44:19,5.0,parent-5.0.0-alpha.1,2021-01-30 18:36:37,0 days 01:07:42,1.0
898,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",2021-01-30 18:36:37,4.0,parent-4.9.1,2021-01-30 18:09:57,0 days 00:26:40,0.0
900,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",2021-01-30 18:09:57,3.0,parent-4.10.0-RC1,2020-10-07 03:28:26,115 days 14:41:31,2774.0
899,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",2020-10-07 03:28:26,2.0,parent-4.9.0,2020-09-11 21:08:20,25 days 06:20:06,606.0
901,Okhttp,https://github.com/square/okhttp,parent-4.9.0,"Commit(sha=""cbeaf8f955fff9caa5652ccc6c1393ec8b...",2020-09-11 21:08:20,1.0,NaN,2020-09-11 21:08:20,0 days 00:00:00,0.0


In [10]:
refactoring_miner_filename

NameError: name 'refactoring_miner_filename' is not defined

In [ ]:
f.close()

In [ ]:
g = Github(github_oauth, timeout=3000)
print(current_project.iloc[0]['project_link'].replace('https://github.com/',''))
repo = g.get_repo(current_project.iloc[0]['project_link'].replace('https://github.com/',''))
print(repo)

In [ ]:
with open('raw_refactoringMiner/Okhttp/Okhttp_parent-5.0.0-alpha.1.json') as f:
    refactoring_miner = json.load(f)
    
for i in refactoring_miner['commits']:

    if len(i['refactorings']) != 0:
        commit_compared_with = i['sha1']
        file_arr = []
        num_line_affected = 0
        num_dependency = 0
        actual_num_of_classes_touched = 0
        for refactor in i['refactorings']:
            #print(refactor)
            for file in refactor['rightSideLocations']:

                #file_arr.append(file['filePath'])
                #print(file['filePath'], 'here')
                #print(file_dict[file['filePath']])
                actual_num_of_classes_touched += 1
                num_dependency += final_dependency_df[file_dict[file['filePath']]]
                num_line_affected += file['endLine'] - file['startLine']
               
        if num_dependency != 0 and num_line_affected != 0:
            main_data_arr.append([latest_commit, commit_compared_with, num_dependency, num_line_affected, actual_num_of_classes_touched])



In [ ]:
main_data_arr

In [ ]:
refactoring_miner_filename

In [ ]:
file_dict

In [ ]:
counter